# Chapter06: Advanced RAG

In [5]:
from dotenv import load_dotenv

loaded = load_dotenv()

## 6.1 Simple RAG Implementation

In [6]:
from langchain_community.document_loaders import GitLoader

def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter
)

documents = loader.load()
print(len(documents))

419


In [8]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# Load the embedding model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# Vectorize text
db = Chroma.from_documents(documents, embeddings)
# Instantiate a retriever
retriever = db.as_retriever()

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈："""
{context}
"""
                                
質問：{question}
''')

model = ChatOpenAI(model="gpt-4o", temperature=0)

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | prompt | model | StrOutputParser()

chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLMs）を活用したアプリケーションを開発するためのフレームワークです。LangChainは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、開発、プロダクション化、デプロイメントの各ステージで役立ちます。\n\n- **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を利用してアプリケーションを構築できます。また、LangGraphを使用して、ストリーミングや人間の介入をサポートする状態保持エージェントを構築できます。\n- **プロダクション化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n- **デプロイメント**: LangGraph Platformを使用して、LangGraphアプリケーションをプロダクション対応のAPIやアシスタントに変換できます。\n\nLangChainは、LLMsや関連技術（埋め込みモデルやベクトルストアなど）の標準インターフェースを実装し、数百のプロバイダーと統合しています。これにより、開発者は異なるプロバイダー間での切り替えが容易になり、複数のコンポーネントを組み合わせた複雑なアプリケーションの構築が可能になります。'

## 6.2 HyDE (Hypothetical Document Embeddings)

We want to retrieve contexts that are relevant to the model’s answer rather than to the query. Therefore, we **first prompt the LLM to generate a hypothetical**.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈："""
{context}
"""
                                
質問：{question}
''')

In [ ]:
hypothetical_prompt = ChatPromptTemplate.from_template("""\
次の質問に回答する一文を書いてください。

質問：{question}
""")

hypothetical_chain = hypothetical_prompt | model | StrOutputParser()

In [ ]:
hyde_rag_chain = {
    "question": RunnablePassthrough(),
    "context": hypothetical_chain | retriever,  # The hypothetical will be sent to `prompt`
} | prompt | model | StrOutputParser()

hyde_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLMs）を活用したアプリケーションを開発するためのフレームワークです。以下のような特徴があります：\n\n1. **開発の簡素化**: LangChainは、オープンソースのコンポーネントやサードパーティの統合を利用して、アプリケーションの開発を簡素化します。LangGraphを使用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **プロダクション化**: LangSmithを使用して、アプリケーションを検査、監視、評価することで、継続的に最適化し、自信を持ってデプロイできます。\n\n3. **デプロイメント**: LangGraphプラットフォームを使用して、LangGraphアプリケーションをプロダクション対応のAPIやアシスタントに変換します。\n\nLangChainは、LLMsや関連技術（埋め込みモデルやベクトルストアなど）の標準インターフェースを実装し、数多くのプロバイダーと統合しています。これにより、開発者は異なるプロバイダー間での切り替えが容易になり、複雑なアプリケーションのオーケストレーションや観察性、評価をサポートします。'

We can also guide model to generate multiple hypotheticals (queries), which can be utilized to retrieve relevant documents.

In [ ]:
'''
`BaseModel` is used to declare exactly what shape the model's output should take.
`Field` is used to regularize the properties in `BaseModel`.
'''

from pydantic import BaseModel, Field

class QueryGenerationOutput(BaseModel):
    queries: list[str] = Field(..., description="検索クエリのリスト")

query_generation_prompt = ChatPromptTemplate.from_template("""\
質問に対してベクターデータベースから関連文書を検索するために、
三つの異なる検索クエリを生成してください。
距離ベースの類似性検索の限界を克服するために、
ユーザーの質問に対して複数の視点を提供することが目標です。
                                                           
質問：{question}
""")

query_generation_chain = (
    query_generation_prompt
    | model.with_structured_output(QueryGenerationOutput)   # Fix the format of output
    | (lambda x: x.queries)     # Get `queries` from `QueryGenerationOutput` object; final output will be a list[str]
)

In [ ]:
multi_query_rag_chain = (
    {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map(), # `retriever.map()` is used to receive a list; list[str] -> list[list[Document]]
    }
    | prompt | model | StrOutputParser()
)
multi_query_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLMs）を活用したアプリケーションを開発するためのフレームワークです。LangChainは、LLMアプリケーションのライフサイクル全体を簡素化することを目的としています。具体的には、以下の3つのステージで役立ちます：\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を利用してアプリケーションを構築できます。また、LangGraphを使用して、ストリーミングや人間の介入をサポートする状態を持つエージェントを構築できます。\n\n2. **プロダクション化**: LangSmithを使用して、アプリケーションを検査、監視、評価することで、継続的に最適化し、自信を持ってデプロイできます。\n\n3. **デプロイメント**: LangGraphアプリケーションをプロダクション対応のAPIやアシスタントに変換できます。\n\nLangChainは、チャットモデルや埋め込みモデル、ベクトルストアなどの大規模言語モデルと関連技術のための標準インターフェースを実装し、数多くのプロバイダーと統合しています。'

We can also use `RAG-Fusion` menthod to determine the rank of retrieved context.

In [ ]:
from langchain_core.documents import Document

def reciprocal_rank_fusion(retriever_outputs: list[list[Document]], k: int = 60) -> list[str]:
    # `{content: score} dict`` for ranking; rank the content by coresponding the score
    content_score_mapping = {}

    for docs in retriever_outputs:
        for rank, doc in enumerate(docs): 
            content = doc.page_content
            # If this context occurs firstly, add it into the `dict`
            if content not in content_score_mapping:
                content_score_mapping[content] = 0
            # accumulate score corresponding to the current context
            content_score_mapping[content] += 1 / (rank + k)
    # rank each context by their accumulated scores
    ranked = sorted(content_score_mapping.items(), key=lambda x: x[1], reverse=True)
    return [content for content, _ in ranked]

In [ ]:
rag_fusion_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map() | reciprocal_rank_fusion,
} | prompt | model | StrOutputParser()

rag_fusion_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLMs）を活用したアプリケーションを開発するためのフレームワークです。LangChainは、LLMアプリケーションのライフサイクル全体を簡素化することを目的としています。具体的には、以下のような機能を提供しています：\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を利用してアプリケーションを構築できます。また、LangGraphを使用して、ストリーミングや人間の介入をサポートするエージェントを構築することができます。\n\n2. **プロダクション化**: LangSmithを使用して、アプリケーションを検査、監視、評価することで、継続的に最適化し、自信を持ってデプロイすることができます。\n\n3. **デプロイ**: LangGraphアプリケーションをプロダクション対応のAPIやアシスタントに変換することができます。\n\nLangChainは、チャットモデルや埋め込みモデル、ベクトルストアなどの関連技術のための標準インターフェースを実装しており、数多くのプロバイダーと統合されています。これにより、開発者は異なるプロバイダー間で簡単に切り替えたり、コンポーネントを組み合わせたりすることができます。'

## 6.3 Cohere

### `Cohere` can be used to rerank documents corresponding to one query.

In [ ]:
from langchain_community.document_loaders import GitLoader

def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter
)

documents = loader.load()
print(len(documents))

419


In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# Load the embedding model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# Vectorize text
db = Chroma.from_documents(documents, embeddings)
# Instantiate a retriever
retriever = db.as_retriever()

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈："""
{context}
"""
                                
質問：{question}
''')

model = ChatOpenAI(model="gpt-4o", temperature=0)

In [ ]:
from typing import Any

from langchain_cohere import CohereRerank
from langchain_core.documents import Document

def rerank(inp: dict[str, Any], top_n: int = 3) -> list[Document]:
    question = inp["question"]
    documents = inp["documents"]

    cohere_reranker = CohereRerank(model="rerank-multilingual-v3.0", top_n=top_n)

    return cohere_reranker.compress_documents(documents=documents, query=question)

rerank_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "documents": retriever,
    }
    | RunnablePassthrough.assign(context=rerank)
    |prompt | model | StrOutputParser()
)

rerank_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLMs）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能を提供しています：\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を利用してアプリケーションを構築できます。また、LangGraphを使用して、ストリーミングや人間の介入をサポートする状態を持つエージェントを構築できます。\n\n2. **プロダクション化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイメント**: LangGraph Platformを使用して、LangGraphアプリケーションをプロダクション対応のAPIやアシスタントに変換できます。\n\nLangChainは、LLMや関連技術（埋め込みモデルやベクトルストアなど）の標準インターフェースを実装し、数百のプロバイダーと統合しています。これにより、開発者は多様なツールやサービスと連携しやすくなっています。'

## 6.4 Attempt to use several `retriever`

Sometimes we want to retrieve different type of documents like LangChain's official document or WEB.

In [ ]:
from langchain_community.retrievers import TavilySearchAPIRetriever

# Instantiate a `langchain document retriever`
langchain_document_retriever = retriever.with_config(
    {"run_name": "langchain_document_retriever"}
)

# Instantiate a `web api retriever` and allow it to retrieve 3 relevant documents
web_retriever = TavilySearchAPIRetriever(k=3).with_config(
    {"run_name": "web_retriever"}
)

In [ ]:
from enum import Enum
from pydantic import BaseModel

class Route(str, Enum):
    langchain_document = "langchain_document"
    web = "web"

class RouteOutput(BaseModel):
    '''
    This `class` is used to fix the format of model's output
    '''
    route: Route

route_prompt = ChatPromptTemplate.from_template("""\
質問に回答するために適切なRetrieverを選択してください。
質問：{question}
""")

route_chain = (
    route_prompt
    | model.with_structured_output(RouteOutput)
    | (lambda x: x.route)
)

In [ ]:
def routed_retriever(inp: dict[str, Any]) -> list[Document]:
    question = inp["question"]
    route = inp["route"]

    if route == Route.langchain_document:
        return langchain_document_retriever.invoke(question)
    elif route == Route.web:
        return web_retriever.invoke(question)
    
    raise ValueError(f"Unkonw route: {route}")

route_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "route": route_chain,
    }
    # `RunnablePassthrough.assign()` is used to pass `question` and `route` arguments to `routed_retriever` function.
    | RunnablePassthrough.assign(context=routed_retriever)
    | prompt | model | StrOutputParser()
)

route_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLMs）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクル全体を簡素化することを目的としています。具体的には、以下の3つのステージでサポートを提供します：\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を利用してアプリケーションを構築できます。また、LangGraphを使用して、ストリーミングや人間の介入をサポートする状態保持エージェントを構築することができます。\n\n2. **プロダクション化**: LangSmithを使用して、アプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイメント**: LangGraph Platformを使用して、LangGraphアプリケーションをプロダクション対応のAPIやアシスタントに変換できます。\n\nLangChainは、LLMや関連技術（埋め込みモデルやベクトルストアなど）の標準インターフェースを実装し、数百のプロバイダーと統合しています。これにより、開発者は迅速にアプリケーションを構築し、デプロイすることが可能です。'

In [ ]:
route_rag_chain.invoke("東京の今日(2025年7月29日)の天気は?")

'2025年7月29日の東京の天気は「曇時々晴」で、気温は最高32℃、最低25℃です。'

## 6.5 Hybrid Retrieve

In [ ]:
from langchain_community.retrievers import BM25Retriever

chroma_retriever = retriever.with_config(
    {"run_name": "chroma_retriever"}
)

bm25_retriever = BM25Retriever.from_documents(documents).with_config(
    {"run_name": "bm25_retriever"}
)

In [ ]:
from langchain_core.runnables import RunnableParallel

hybrid_retriever = (
    RunnableParallel({
        "chroma_documents": chroma_retriever,
        "bm25_documents": bm25_retriever,
    })
    | (lambda x: [x["chroma_documents"], x["bm25_documents"]])
    | reciprocal_rank_fusion
)

In [ ]:
hybrid_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": hybrid_retriever,
    }
    | prompt | model | StrOutputParser()
)

hybrid_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLMs）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクル全体を簡素化することを目的としています。具体的には、以下の3つのステージでサポートを提供します：\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を利用してアプリケーションを構築できます。また、LangGraphを使用して、ストリーミングや人間の介入をサポートするステートフルなエージェントを構築することができます。\n\n2. **プロダクション化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイメント**: LangGraph Platformを使用して、LangGraphアプリケーションをプロダクション対応のAPIやアシスタントに変換できます。\n\nLangChainは、LLMや関連技術（埋め込みモデルやベクトルストアなど）の標準インターフェースを実装し、数百のプロバイダーと統合しています。これにより、開発者は多様なツールやサービスを活用して、より強力で柔軟なアプリケーションを構築することができます。'